# Math 34A Spring 2022 Grading Script

## Import Libraries and Functions

In [1]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from grading_functions import *
from matplotlib import pyplot as plt

## Import Grade Files

In [2]:
"""grades is from Gradescope, 
roster is the current student list from egrades, 
webwork is the homework total downloaded from Gauchospace"""
grades = pd.read_csv("Math_34A_Spring_2022_Grades.csv")
roster = pd.read_csv("Math_34A_Spring_2022_Roster.csv")
webwork = pd.read_csv("Math_34A_Spring_2022_Webwork.csv", na_values=["-"])

## Creating a Quiz Max Points List
The list will be used later to adjust quiz grades for students who were sick and missed section. 

In [3]:
#Total Quiz Points Column
quiz_point_max_columns = [name for name in grades.columns if "Max Points" in name and "quiz" in name.lower()]
#grades["QPTot"] = grades.apply(lambda row: sum([row[name] for name in quiz_point_max_columns]), axis=1)
quiz_max_points = grades[quiz_point_max_columns].iloc[1].values

## Basic Cleaning
1 Renaming columns in all three dataframes for compatability and ease of reading

2 Deleting garbage columns from Gradescope's list

3 Slimming down the egrades roster and webwork roster to include only what we need for student identification

In [4]:
#Renameing "Perm #" to "SID" in Egrades, which may not even be necessary. 
roster.rename(columns = {"Perm #": "SID"}, inplace = True)

#Renaming the same and the Homework column in Webwork
webwork.rename(columns = {"ID number": "SID"}, inplace = True)
webwork.rename(columns = {"External tool:Webwork (Percentage)": "HWT"}, inplace = True)

#Deleting Garbage Columns
junk_column_indices = [i for i, column in enumerate(grades.columns) \
                       if "Max Points" in column or "Submission Time" in column \
                       or "Lateness" in column or column == "section_name" or column == "Email"]
grades.drop(grades.columns[junk_column_indices], axis=1, inplace=True)

#Renaming Columns: Homework and quizzes (In grades df)
quiz_column_names = [name for name in grades.columns if "quiz" in name.lower()]
grades.rename(columns = {name: "Quiz"+" "+str(i+1) for i, name in enumerate(quiz_column_names)}, inplace = True)


#Roster and Webwork Operations
roster = roster[["Enrl Cd", "SID"]]
webwork = webwork[["SID", "HWT"]]

#Cleaning Webwork's Homework Total Column because there are % symbols, making it an object. 
#Note: The commented-out method on two lines below worked, but on the line below it I tried using methods I learned 
#from Codecademy on 6/30 that also work but with a simpler regex command and a more robust .to_numeric pandas method. 
#webwork["HWT"] = webwork["HWT"].str.extract('([0-9.]+)').astype(float)
webwork["HWT"] = pd.to_numeric(webwork["HWT"].str.replace("[\%]", "", regex=True))

## Merging the two versions of Midterm 1
They are merged into a new column  called "Midterm 1" and dropped once this is done

In [5]:
midterm1_version_list = ["Midterm 1 Gold", "Midterm 1 Blue"]
grades["Midterm 1"] = grades.apply(version_merger, 
                                   version_list=midterm1_version_list, 
                                   axis=1)
#grades["Midterm 1"] = grades.apply(lambda row: version_merger(row, midterm1_version_list), axis=1)
grades.drop(columns=midterm1_version_list, axis=1, inplace=True)

## Merging the Three Dataframes

In [6]:
#merging Gradescope and Egrades
grades_merge_one = pd.merge(grades, roster, on="SID", how ="right")
#merging the result and Webwork
grades_for_submission = pd.merge(webwork, grades_merge_one, on="SID", how ="right")

## More pruning of columns
I'm not sure if this is necessary because it looks like the same was done for the Gradescope file. Maybe it's also done for the egrades list - and this could be done at the beginning. 

In [7]:
#Pruning more columns from general roster
junk_column_indices = [i for i, column in enumerate(grades_for_submission.columns) \
                       if "Pronouns" in column or "Lateness" in column \
                       or column == "section_name" or column == "Email"]
grades_for_submission.drop(grades_for_submission.columns[junk_column_indices], axis=1, inplace=True)

## Scaling the Midterm Scores to make them out of 100. 
Their column names are also shortened to make the spreadsheet easier to read when you open the file. 

In [8]:
#Scaling the Midterms
grades_for_submission["M1"] = np.round(grades_for_submission["Midterm 1"]*4, decimals=1)
grades_for_submission["M2"] = np.round(100*grades_for_submission["Midterm 2"]/23, decimals=1)
grades_for_submission["M3"] = np.round(100*grades_for_submission["Midterm 3"]/40, decimals=1)
#Drop Midterm Point Columns
grades_for_submission.drop(columns=["Midterm 1", "Midterm 2", "Midterm 3"], axis=1, inplace=True)

## Adjusting Quiz Grades to account for excused absences. 

### Part 1 
Getting a list of all of Trevor's students so all of their quiz 4's and quiz 6's can be excused. 

In [9]:
#Missing Quiz Corrections
#Trevor's cover left all of the quizzes out on a table in the tea room, 
#so here is a list of all of his students. 
trevor_df = pd.read_csv("Excused Tests/Trevor's Students.csv")
trevor_students = list(trevor_df["SID"])

## Adjusting Quiz Grades to account for excused absences.

### Part 2 
Constructing a dataframe to hold information in the place where a student has an excused abscence from a quiz: The row/column corresponding to that quiz score for that student has a tuple. Students with excused absences have a (0,max_points) tuple and every other entry has a (score, 0) tuple. The quiz average column of the dataframe is then computed using these tuples: We sum over the scores of the first entries (giving the total quiz points earned) and divided by the total possible quiz points minus the individual max_point values for quizzes that are excused. This computes the same total as if the students had their excused quizzes replaced by their quiz average, and we'll use this fact at the bottom of the cell below (though it's not needed for grade calculation). 

In [10]:
missing_quiz_matrix = grades_for_submission[["SID"]].copy()
#excused_list_points
n=len(quiz_max_points)
for i in range(1,n+1):
    quiz_df = pd.read_csv("Excused Tests/Quiz "+str(i)+" Excused Students.csv")
    excused_list = list(quiz_df["SID"])
    missing_quiz_matrix["Quiz "+str(i)+"-"] = grades_for_submission.fillna(0).apply(quiz_tuple_function, 
                                                                                    excused_list = excused_list, 
                                                                                    trevor_students = trevor_students, 
                                                                                    i = i, 
                                                                                    quiz_max_points = quiz_max_points, 
                                                                                    axis=1)

    

#Computing the Quiz Total (average percentage score)
grades_for_submission["QT"] = missing_quiz_matrix.apply(quiz_average_calculator, 
                                                        i = i, 
                                                        quiz_max_points = quiz_max_points, 
                                                        axis=1)

#Just for fun, this replaces the excused missing quiz score with quiz total percent of max points, so if you add the
#entries in Excel, you should get the same quiz total for these students. I just tried it in Excel. They agree up to 
#a rounding error. 
for i in range(1,len(quiz_max_points)+1):
    quiz_df = pd.read_csv("Excused Tests/Quiz "+str(i)+" Excused Students.csv")
    excused_list = list(quiz_df["SID"])
    grades_for_submission["Quiz "+str(i)] = grades_for_submission.apply(quiz_missing_points_allocator, 
                                                                        quiz_max_points = quiz_max_points, 
                                                                        i = i, 
                                                                        excused_list = excused_list, 
                                                                        trevor_students = trevor_students, 
                                                                        axis=1
                                                                       )

## Sorting the Remaining Columns
If I open a CSV file of the df now, columns are everywhere! This block sorts the remaining columns and puts the key ones in the front. 

The dataframe with grades on it is also shortened from grades_for_submission to gfs since I copied it. 
The copy was probably not necessary, but I was new to the practice. 

In [11]:
#Let's sort the columns and shorten the name of the data set (so gfs will be grades_for_submission sorted)
gfs_copy = grades_for_submission.copy()
#first_columns = ["First Name", "Last Name", "Enrl Cd", "SID", "Grade", "ClassLevel", "Major1", "Major2"]
first_columns = ["First Name", "Last Name", "Enrl Cd", "SID"]
gfs_copy.drop(first_columns, axis=1, inplace=True)
gfs_copy = gfs_copy[sorted(gfs_copy.columns)]
gfs = gfs_copy.copy()
for i, info_col in enumerate(first_columns):
    gfs.insert(i, info_col, grades_for_submission[info_col], True)

## New Columns: QT and QTA
Creating a Quiz Total column and an Adjusted Quiz Total column to account for the Quiz Reflections: x% of half their missing points should be added, where x is their score on the corresponding "Reflection for Improvement" assignment on Gradescope. For example, if they reflected on 100% of their missed questions, then half of the difference between their quiz average and 100% would be added to their quiz total. For example, an initial quiz total of 40% would be boosted to 70% for a student who completed the reflection. 

### Remark: 
In Gradescope I named the reflection assignment "Q Reflections..." instead of "Quiz Reflections..." because otherwise the script above would have renamed it something like "Quiz 11." Clearly, we don't want that. 

In [12]:
#Quiz Total Column
quiz_list=["Quiz "+str(i) for i in range(1,len(quiz_column_names)+1)]
missing_quiz_point_multiplier = gfs["Q Reflection for Improvement"].fillna(0)/200
gfs["QTA"] = gfs["QT"] + (100-gfs["QT"])*missing_quiz_point_multiplier

## Final Exam Column
Converting the "Final Exam" raw score into a "Final" entry with a percent value. 

In [13]:
final_point_total = 72
gfs["Final"] = np.round(gfs["Final Exam"]*100/final_point_total, decimals=1)

## Replacing Excused Missing Midterms 
These will be replaced with the final percentage (that was just calculated). 

In [14]:
#Replacing Midterm Entries for Excused Absences (that midterm is replaced w/ the final score)
for i in range(1,4):
    excused_df = pd.read_csv("Excused Tests/Midterm "+str(i)+" Excused Students.csv")
    students_list = list(excused_df["SID"])
    gfs["M"+str(i)] = gfs.apply(lambda row: np.round(missed_exam_replacer(row,
                                                                             students_list, 
                                                                            "M"+str(i), 
                                                                            "Final"), 
                                                    decimals=1),
                                axis=1)

#Final Replacement will be done below because we need to replace the final with the midterm average

## Computing the Midterm Average
MT or Midterm Total is the raw percentage as the average of the two highest midterms, which is congruent with making the third midterm optional so students could take a third midterm if they wanted to improve their midterm grade. In practice, I added the three midterm percentages, subtracted dropped the minimum, and then divided by two. I could not find a way to just "drop the lowest," though I strongly suspect there is a numpy or Pandas method for this. 

MTA is the adjusted midterm average, with x% of the missing points returned based on the x% completed for the midterm reflections assignment. 

In [15]:
#Two New Midterm Grade Columns (Raw Average and "MT" that drops the lowest midterm) -- I dropped "Raw Average" below
gfs["Raw Midterm Average"] = gfs.apply(lambda row: (row.fillna(0)["M1"]+row.fillna(0)["M2"]+row.fillna(0)["M3"])/3
                                                                     , axis = 1)
gfs["MT"] = gfs.apply(lambda row: (3*row["Raw Midterm Average"] - np.min([row.fillna(0)["M1"],row.fillna(0)["M2"],
                                                                          row.fillna(0)["M3"]]))/2, axis = 1)
#Adjusting the Midterm Total to include missing points
missing_midterm_point_multiplier = gfs["Exam Reflection for Improvement"].fillna(0)/200
gfs["MTA"] = gfs["MT"] + (100-gfs["MT"])*missing_midterm_point_multiplier
gfs.drop(columns=["Raw Midterm Average"], axis=1, inplace=True)

## Excused Absence Replacement
There were some students who had documented sickness during the final exam. The code below used the list of SIDs sent to me by my TAs and based on that list, replaced those students' final scores with their adjusted midterm average to avoid them having to take the course as an incomplete. 

In [16]:
#Replacing Final Entries for Excused Absences (that midterm is replaced w/ the final score)
final_df = pd.read_csv("Excused Tests/Final Exam Excused Students.csv")
final_missed = list(final_df["SID"])
gfs["Final"] = gfs.apply(lambda row: missed_exam_replacer(row, final_missed, "Final", "MTA"), 
                                axis=1)
#gfs["Final na0"] = gfs["Final"].fillna(0)

## Computing the "Grade" and "Letter Grade" columns, now that we're ready. 

In [17]:
#Grade Column
gfs["Grade"] = gfs.apply(lambda row: .1*row["QTA"] + .2*row["HWT"] + .4*row["MTA"] + .3*row.fillna(0)["Final"] + 3,
                        axis=1)
   

#Letter Grade Column
gfs["Letter Grade"] = gfs.apply(lambda row: letter_grade_assigner(row), axis=1)


#ordering columns for Egrades Submission and my sanity
first_columns = ["SID", "Enrl Cd", "Letter Grade", "Final", "MTA", "HWT", "QTA"]
temp_columns = gfs[first_columns]
gfs.drop(columns=first_columns, axis=1, inplace=True)
for (i, col_name) in enumerate(first_columns):
    gfs.insert(i, col_name, temp_columns[col_name])

## Saving the finished dataframe as a CSV file. 

In [18]:
#Save gfs df to a file
gfs.to_csv('Math_34A_Winter_2022_Grades_python_output.csv', header=True, index=False)

## Creating a slimmer CSV file for Uploading to Egrades

In [19]:
##Slim Format for Submission
"Enrl Cd", "SID"
gfs_slim = gfs[["Enrl Cd", "SID", "Letter Grade"]]
gfs_slim.to_csv('Math 34A Egrades List Spring 22.csv', header=True, index=False)